In [21]:
import tkinter as tk
from tkinter import filedialog

In [22]:
class Task:
    def __init__(self, index, duration, name, prereq_numbers):
        self.index = index
        self.duration = duration
        self.name = name
        self.prereq_numbers = prereq_numbers
        
    def __str__(self):
        return self.name
    
    def numbers_to_tasks(self, tasks):
        self.prereq_tasks = []
        for number in self.prereq_numbers:
            self.prereq_tasks.append(tasks[number])
            
    def set_times(self):
        self.start_time = max([p.end_time for p in self.prereq_tasks], default=0)
        self.end_time = self.start_time + self.duration
        
    def mark_is_critical(self):
        self.is_critical = True
        for prereq_task in self.prereq_tasks:
            if prereq_task.end_time == self.start_time:
                prereq_task.mark_is_critical()


In [23]:
class PoSorter:
    def __init__(self):
        self.tasks: list[Task] = []
    
    def prepare_tasks(self):
        # Initialise the tasks
        for task in self.tasks:
            task.followers = []
            task.is_critical = False
        # Give each task a followers list that holds references to the tasks that
        # must follow it. i.e., this task is a pre req for tasks in the followers list.
        for task in self.tasks:
            for prereq_task in task.prereq_tasks:
                prereq_task.followers.append(task)
            task.prereq_count = len(task.prereq_tasks)
        
    def build_pert_chart(self):
        self.prepare_tasks()
        
        ready_tasks = []
        
        # Add tasks with no prerequisites to ready_tasks
        for task in self.tasks:
            if task.prereq_count == 0:
                ready_tasks.append(task)
        
        # Organise tasks in columns where pre-requisite tasks are shown on columns to the left
        # of tasks that depend on them. Tasks are also organised in batches (appearing in the same
        # column) where they can be processed independent of each other.
        self.columns = []
        batch_number = 0
        new_ready_tasks = []
        
        while ready_tasks:
            task = ready_tasks.pop(0)
            task.set_times()
            if len(self.columns) <= batch_number:
                self.columns.append([])
            self.columns[batch_number].append(task)
            
            # Loop through task's followers
            for follower in task.followers:
                # Decrement the follower's prereq_count
                follower.prereq_count -= 1
                # if the follower's prereq_count is now 0, add it to the ready_task
                if follower.prereq_count == 0:
                    new_ready_tasks.append(follower)
            
            # when ready_tasks is empty, set it equal to new_ready_tasks,
            # set new_ready_tasks equal to a new empty list, and continue.
            if not ready_tasks:
                ready_tasks = new_ready_tasks
                batch_number += 1
                new_ready_tasks = []
                
        # find the finish or last task and mark it as critical
        finish_tasks = self.columns[-1]
        if finish_tasks:
            finish_tasks[0].mark_is_critical()
    
    def draw_pert_chart(self, canvas):
        spacing = 10                        # Margin for the first task starting from the left
                                            # and spacing between tasks in a column.
        column_spacing = 50                 # Spacing separating tasks between columns.
        box_x_size = 55                     # Sizes of the rectangle representing a task
        box_y_size = 85
        box_half_width = box_x_size / 2
        box_half_height = box_y_size / 2
        
        # Set the Task objects centre, where each rectangle and text should be drawn
        for column in range(len(self.columns)):
            tasks = self.columns[column]
            for task in range(len(tasks)):
                tasks[task].centre = (
                    spacing + (column_spacing + box_x_size) * column + box_half_width,
                    (spacing + box_y_size) * task + spacing + box_half_height
                )

        # Clear any previous drawing.
        canvas.delete('all')
        
        # Loop through the tasks to draw the lines with arrows
        for task in self.tasks:
            try:
                hasattr(task, 'centre')
            
                # draw arrow from left edge of each task to the right edges of its prerequisites
                for prereq_task in task.prereq_tasks:
                    is_link_critical = prereq_task.end_time == task.start_time
                    is_critical_to_project = is_link_critical and task.is_critical
                    canvas.create_line(
                        task.centre[0] - box_half_width,
                        task.centre[1],
                        prereq_task.centre[0] + box_half_width,
                        prereq_task.centre[1],
                        arrow=tk.FIRST,
                        width=3 if is_link_critical else 1,
                        fill='red' if is_critical_to_project else 'black'
                    )
            except AttributeError: continue
        
        # Loop through the tasks to draw the rectangles and task indexes as the text
        for task in self.tasks:
            try:
                hasattr(task, 'centre')

                canvas.create_rectangle(
                    task.centre[0] - box_half_width,
                    task.centre[1] - box_half_height,
                    task.centre[0] + box_half_width,
                    task.centre[1] + box_half_height,
                    fill='pink' if task.is_critical else 'light blue',
                    outline='red' if task.is_critical else 'black'
                )
                text = f"""Task {task.index}\nDur: {task.duration}\nStart: {task.start_time}\nEnd: {task.end_time}"""
                canvas.create_text(
                    task.centre, 
                    text=text, 
                    fill='red' if task.is_critical else 'black'
                )
            except AttributeError: continue
        
    def topo_sort(self):
        self.prepare_tasks()
        
        ready_tasks = []
        
        # Add tasks with no prerequisites to ready_tasks
        for task in self.tasks:
            if task.prereq_count == 0:
                ready_tasks.append(task)

        self.sorted_tasks = []
        
        while ready_tasks:
            task = ready_tasks.pop(0)
            self.sorted_tasks.append(task)
            
            # Loop through task's followers
            for follower in task.followers:
                # Decrement the follower's prereq_count
                follower.prereq_count -= 1
                # if the follower's prereq_count is now 0, add it to the ready_task
                if follower.prereq_count == 0:
                    ready_tasks.append(follower)
    
    def verify_sort(self):
        tasks_successfully_sorted = 0
        for task in self.sorted_tasks:
            if all(i < self.sorted_tasks.index(task) for i in [self.sorted_tasks.index(p) for p in task.prereq_tasks]):
                tasks_successfully_sorted += 1
            else:
                return f"Task {task} should not come before any of its pre-requisite tasks."
        return f"Successfully sorted {tasks_successfully_sorted} out of {len(self.tasks)} tasks."
    
    @staticmethod
    def read_task(file_handle) -> Task:
        import re
        while task_line := file_handle.readline():
            pattern = re.compile('^\s*([\d]+),\s*([\d]+),([^,]+),\s*\[(.*)\]\s*$')
            matcher = pattern.match(task_line)
            if matcher:
                index = matcher.group(1).strip()
                duration = int(matcher.group(2).strip())
                task_name = matcher.group(3).strip()
                prereq_numbers = matcher.group(4).strip().split(',')
                prereq_numbers = [int(n) for n in prereq_numbers if n]
                return Task(index, duration, task_name, prereq_numbers)
    
    def load_po_file(self, filename):
        self.tasks = []
        with open(filename) as file_handle:
            while task := self.read_task(file_handle):
                self.tasks.append(task)
                
        for task in self.tasks:
            task.numbers_to_tasks(self.tasks)


In [24]:
class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.sorter = PoSorter()
        
        # Make the main interface
        self.window = tk.Tk()
        self.window.title('draw_critical_paths')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('400x300')
        
        # Build the menu.
        self.menubar = tk.Menu(self.window)
        self.menu_file = tk.Menu(self.window, tearoff=False)
        self.menu_file.add_command(label='Open...', command=self.open_po, accelerator='Ctrl+O')
        self.menu_file.add_separator()
        self.menu_file.add_command(label='Exit', command=self.kill_callback)
        self.menubar.add_cascade(label='File', menu=self.menu_file)
        self.window.config(menu=self.menubar)
        
        # Build the canvas.
        self.canvas = tk.Canvas(self.window, borderwidth=2, relief=tk.SUNKEN, bg='white')
        self.canvas.pack(padx=10, pady=(0, 10), fill=tk.BOTH, expand=True)
        
        self.window.bind('<Control-o>', self.ctrl_o_pressed)
        
        # Display the window.
        self.window.focus_force()
        self.window.mainloop()
        
    def kill_callback(self):
        self.window.destroy()
        
    def ctrl_o_pressed(self, event):
        self.open_po()
    def open_po(self):
        file_types = [('Partial Ordering', '*.po')]
        filename = filedialog.askopenfilename(
            defaultextension='.po',
            filetypes=file_types,
            initialdir='.',
            title='Open Partial Ordering'
        )
        if not filename: return
        
        self.sorter.load_po_file(filename)
        self.sorter.build_pert_chart()
        self.sorter.draw_pert_chart(self.canvas)


In [25]:
App()